---


**Importando as bibliotecas**

---



In [21]:
# coleta, tratamento e manipulação dos dados
import numpy as np
import pandas as pd

# visualização e geração de mapas de calor
import folium
from folium import plugins
import branca.colormap as cm

# visualização e geração dos gráficos
import matplotlib.pyplot as plt
from datetime import time

# **1. Coletar dados**

---

**Coleta dos dados**: acidentes nas rodovias federais nordestinas em 2018

---



In [ ]:
#Lê o dataset de acidentes no Brasil armazenado no arquivo csv
df = pd.read_csv("datatran2018.csv", encoding='latin', sep=";", error_bad_lines=False)

#Cria um filtro para selecionar apenas acidentes no Nordeste
filter_northeast = ((df["uf"] == "BA") | (df["uf"] == "PE") | (df["uf"] == "AL") |
                    (df["uf"] == "MA") | (df["uf"] == "CE") | (df["uf"] == "PB") |
                    (df["uf"] == "PI") | (df["uf"] == "RN") | (df["uf"] == "CE"))

#Atualiza o dataframe apenas com os acidentes no Nordeste
df = df[filter_northeast]

#Mostra o número de linhas (acidentes) e colunas (informações) do dataframe de acidentes
df.shape

Com a coleta de dados pode-se perceber que no ano de 2018, houveram 14285 acidentes nos trechos das rodovias federais que estão situados no nordeste.

# **2. Realizar pré-processamento dos dados**

---

**Definição de tipos**

---



In [ ]:
#Apresenta os cinco primeiros dados do dataframe
df.head()

*   **id**: categórico ordinal
*   **data_inversa**: categórico ordinal
*   **dia_semana**: categórico nominal
*   **horário**: categórico ordinal
*   **uf**: categórico nominal
*   **br**: categórico nominal
*   **km**: contínuo
*   **município**: categórico nominal
*   **causa_acidente**: categórico nominal
*   **tipo_acidente**: categórico nominal
*   **classificacao_acidente**: categórico nominal
*   **fase_dia**: categórico nominal
*   **sentido_via**: categórico nominal
*   **condicao_metereologica**: categórico nominal
*   **tipo_pista**: categórico nominal
*   **tracado_via**: categórico nominal
*   **uso_solo**: categórico binário
*   **pessoas**: contínuo
*   **mortos**: contínuo
*   **feridos_leves**: contínuo
*   **feridos_graves**: contínuo
*   **ilesos**: contínuo
*   **ignorados**: contínuo
*   **feridos**: contínuo
*   **veiculos**: contínuo
*   **latitude**: contínuo
*   **longitude**: contínuo
*   **regional**: categórico nominal
*   **delegacia**: categórico nominal
*   **uop**: categórico nominal

**Categóricos ordinais**: 3

**Categóricos binários**: 1

**Categóricos nominais**: 15

**Contínuos**: 11

---

**Tratamento de dados ausentes**: como apresentado abaixo, 28 amostras não possuem os dados relacionados à BR e ao KM onde ocorreu o acidente. Devido a impossibilidade de inputar estes dados, as amostras foram removidas.

---



In [ ]:
#Verifica a quantidade de NaN em cada coluna do dataset
df.isna().sum()

In [ ]:
#Remove as linhas que possuem NaN em alguma coluna
df.dropna(inplace = True)
#Informa as dimensões do dataset
df.shape

Após a remoção dos dados ausentes, relacionados à BR onde o acidente aconteceu, restaram 14257 acidentes para análise.

---

**Normalização e Discretização**: O processo de normalização consistiu na transformação dos dados 'causa_acidente' e 'tipo_acidente' em níveis de 0 a 1 que indicam do melhor ao pior caso. Em relação ao processo de discretização, foram definidos intervalos que agrupam os acidentes por quantidade pessoas envolvidas a partir do processamento do dado 'pessoas'. Ambos os processos resultaram em novas colunas no dataframe.

---

In [ ]:
#Lista todas as causas de acidentes
df["causa_acidente"].value_counts()

In [ ]:
#Tupla de causas de acidente onde não existe a culpa do condutor
no_driver_irresponsibility = ("Animais na Pista", "Falta de Atenção do Pedestre", "Pista Escorregadia", "Defeito na Via", "Mal Súbito", "Sinalização da via insuficiente ou inadequada", "Agressão Externa", "Desobediência às normas de trânsito pelo pedestre")

#Tupla de causas de acidente onde existe baixa culpa do condutor
low_driver_irresponsibility = ("Restrição de Visibilidade", "Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo", "Fenômenos da Natureza")

#Tupla de causas de acidente onde existe culpa moderada do condutor
moderate_driver_irresponsibility = ("Avarias e/ou desgaste excessivo no pneu", "Objeto estático sobre o leito carroçável", "Carga excessiva e/ou mal acondicionada")

#Tupla de causas de acidente onde existe alta culpa do condutor
high_driver_irresponsibility = ("Não guardar distância de segurança", "Defeito Mecânico no Veículo", "Condutor Dormindo")

#Tupla de causas de acidente onde existe culpa muito alta do condutor
very_high_driver_irresponsibility = ("Falta de Atenção à Condução", "Desobediência às normas de trânsito pelo condutor", "Ingestão de Álcool", "Velocidade Incompatível", "Ultrapassagem Indevida", "Ingestão de álcool e/ou substâncias psicoativas pelo pedestre", "Ingestão de Substâncias Psicoativas")

#Transforma a causa do acidente em um índice que representa o nível de culpa do condutor
def normalize_accident_cause(accident_cause):
  #Variável para o nível de culpa do condutor
  level = None

  #Níveis de culpa
  if accident_cause in no_driver_irresponsibility:
    level = 0.0
  elif accident_cause in low_driver_irresponsibility:
    level = 0.25
  elif accident_cause in moderate_driver_irresponsibility:
    level = 0.5
  elif accident_cause in high_driver_irresponsibility:
    level = 0.75
  elif accident_cause in very_high_driver_irresponsibility:
    level = 1.0

  #Retorna o nível de culpa do condutor
  return level

#Cria uma nova coluna com dado categórico ordinal a partir do cálculo
#do nível de culpa do condutor no acidente
df["causa_acidente_normalizado"] = df["causa_acidente"].apply(normalize_accident_cause)

#Mostra a coluna primitiva e a derivada a ela
df[["causa_acidente", "causa_acidente_normalizado"]].head()

In [ ]:
#Lista todas os tipos de acidentes
df["tipo_acidente"].value_counts()

In [ ]:
#Tupla de acidentes de risco mínimo
very_low_risk = ("Colisão traseira", "Danos eventuais", "Derramamento de carga")

#Tupla de acidentes de baixo risco
low_risk = ("Queda de ocupante de veículo", "Atropelamento de Animal", "Colisão com objeto em movimento")

#Tupla de acidentes de risco moderado
moderate_risk = ("Colisão com objeto estático", "Engavetamento")

#Tupla de acidentes de alto risco
high_risk = ("Colisão transversal", "Tombamento", "Saída de leito carroçável", "Incêndio")

#Tupla de acidentes de risco máximo
very_high_risk = ("Colisão lateral", "Colisão frontal", "Atropelamento de Pedestre", "Capotamento")

#Transforma o tipo do acidente em um índice que representa o nível de risco
def normalize_accident_type(accident_type):
  #Variável para o nível de risco do acidente
  level = None

  #Níveis de risco
  if accident_type in very_low_risk:
    level = 0.0
  elif accident_type in low_risk:
    level = 0.25
  elif accident_type in moderate_risk:
    level = 0.5
  elif accident_type in high_risk:
    level = 0.75
  elif accident_type in very_high_risk:
    level = 1.0

  #Retorna o nível de risco
  return level

#Cria uma nova coluna com dado categórico ordinal a partir do cálculo
#do nível de risco de vítimas no acidente
df["tipo_acidente_normalizado"] = df["tipo_acidente"].apply(normalize_accident_type)

#Mostra a coluna primitiva e a derivada a ela
df[["tipo_acidente", "tipo_acidente_normalizado"]].head()

In [ ]:
#Acidente com o maior número de pessoas envolvidas
df["pessoas"].max()

In [ ]:
#Discretiza o número de pessoas envolvidas nos acidentes em intervalos de 10
def discretize_n_people(people):
  #Variável para o intervalo
  range = None

  #Intervalos considerados a partir do máximo de pessoas envolvidas no dataset
  if people >= 0 and people < 10:
    range = "[0, 10)"
  elif people >= 10 and people < 20:
    range = "[10, 20)"
  elif people >= 20 and people < 30:
    range = "[20, 30)"
  elif people >= 30 and people < 40:
    range = "[30, 40)"
  elif people >= 40 and people < 50:
    range = "[40, 50)"
  elif people >= 50 and people <= 60:
    range = "[50, 60]"

  #Retorna o intervalo calulado para determinado número de pessoas
  return range

#Cria uma nova coluna com dado categórico nominal a partir do cálculo do 
#intervalo de pessoas para cada linha
df["pessoas_discretizadas"] = df["pessoas"].apply(discretize_n_people)

#Mostra a coluna primitiva e a derivada a ela
df[["pessoas", "pessoas_discretizadas"]].head()

---

**Limpeza de dados**: diante da ausência de outliers, incorretos e dados duplicados, a limpeza foi realizada apenas para dados irrelevantes para esta análise: id, km, municipio, sentido_via, feridos, regional, delegacia e uop

---

**ANALISE DE OUTLIERS AQUI**

In [ ]:
#Remove as colunas irrelevantes
df.drop(["id", "km", "municipio", "sentido_via", "feridos", "regional", "delegacia", "uop"], axis = 1, inplace = True)
#Mostra o novo dataframe
df.head()

Após a realização das etapas de pré-processamento, o dataframe ficou na situação apresentada acima.

#**3. Apresentar estatísticas descritivas dos dados com visualizações**

---

**Estatísticas univariadas**: apresenta o panorama da quantidade de pessoas que se envolveram em acidentes nas rodovias nordestinas.

---

In [ ]:
df.groupby(['uf']).describe()['pessoas']

Descrição estatística dos acidentes por estado, tendo como parâmetro a quantidade de pessoas envolvidas. 

In [ ]:
df.groupby(['uf']).mean()['pessoas'].plot.bar(figsize=(20,10))

O gráfico que relaciona a média de pessoas envolvidas nos acidentes por estado apresenta um valor muito próximo para cada estado nordestino.

In [ ]:
df.groupby(['uf']).median()['pessoas'].plot.bar(figsize=(20,10))

In [ ]:
df.mode()['pessoas']

O gráfico que relaciona a mediana de pessoas envolvidas nos acidentes por estado apresenta o mesmo número de pessoas para cada estado. Analisando em conjunto à moda, é possível visualizar que existe uma tendência de envolvimento de duas pessoas nos acidentes destas rodovias federais.

In [ ]:
df.groupby(['uf']).std()['pessoas'].plot.bar(figsize=(20,10))

O gráfico que relaciona o desvio padrão de pessoas envolvidas nos acidentes por estado apresenta uma variação maior entre os estados, em relação aos outros. Tendo Bahia, Maranhão e Piauí com maiores desvios.

In [ ]:
df.groupby(['uf']).max()['pessoas'].plot.bar(figsize=(20,10))

O gráfico que relaciona o máximo de pessoas envolvidas nos acidentes por estado também apresenta uma variação maior entre os estados, em relação aos outros. Neste, é possível visualizar que a Bahia, Maranhão e o Piauí lideram em acidentes envolvendo muitas pessoas. Fato que está de acordo com os dados de desvio padrão. 

---

**Estatísticas bivariadas**: apresenta a correlação entre a quantidade de pessoas que se envolveram em acidentes nas rodovias nordestina, relacionando com o número de óbitos, feridos e ilesos.

---

In [ ]:
df[["pessoas", "mortos", "feridos_leves", "feridos_graves", "ilesos"]].corr(method='pearson')

Correlaciona o número de pessoas envolvidas nos acidentes, trazendo a proporção de mortos, feridos leves e graves e ilesos.

In [ ]:
df.plot.scatter(x='pessoas', y='mortos')

Mostra que os acidentes com menor número de pessoas envolvidas possuem os maiores números de óbitos.

In [ ]:
df.plot.scatter(x='pessoas',y='feridos_leves')

Mostra que os acidentes com menor número de pessoas envolvidas possuem uma maior concentração de feridos leves.

In [ ]:
df.plot.scatter(x='pessoas',y='feridos_graves')

Mostra que os acidentes com menor número de pessoas envolvidas possuem uma maior concentração de feridos graves.


In [ ]:
df.plot.scatter(x='pessoas',y='ilesos')

Mostra que os acidentes com menor número de pessoas envolvidas possuem uma maior concentração de ilesos.

---

**Mapa de calor de acidentes:** apresenta o índice de acidentes de acordo com a localização das incidências 

---

In [ ]:
# lista de coordendas dos acidentes
coordinates = []

# armazena as coordenadas de todos acincentes registrados no dataframe
for latitude, longitude in zip(df.latitude.values, df.longitude.values):    
  coordinates.append([latitude, longitude])

# intancia a visualização do mapa do Nordeste
heatmap_total_accidents = folium.Map(
    location=[-9.231630041999422, -39.87133381935983],
    tiles="Stamen Terrain",
    zoom_start=6,    
)

# adiciona os pontos de calor relacionados ao localização dos acidentes
heatmap_total_accidents.add_child(plugins.HeatMap(coordinates, blur=25))
heatmap_total_accidents

---

**Mapa de acidentes por nível de irresponsabilidade do condutor:** apresenta o íncide de acidentes de acordo com a normalização realizada nos dados referentes a causa do acidente

---

In [ ]:
# lista de coordendas dos acidentes
coordinates_cause_accidents = []

# armazena as coordenadas de todos acincentes e seus respectivos tipos registrados no dataframe
for latitude, longitude, weight in zip(df.latitude.values, df.longitude.values, df.causa_acidente_normalizado.values):    
  coordinates_cause_accidents.append([latitude, longitude, weight])

# intancia a visualização do mapa do Nordeste
heatmap_cause_accidents = folium.Map(
    location=[-9.231630041999422, -39.87133381935983],
    tiles="Stamen Terrain",
    zoom_start=6,    
)

# define a escala de cores para cada nível de responsabilidade
colormap = cm.LinearColormap(colors=['#FDE788', '#FEAF68', '#FE7748', '#FF3F27', '#FF0707'], 
                             index=[0.0, .25, .50, .75, 1.0], 
                             vmin=0.0, 
                             vmax=1.0, 
                             caption='Nível de irresponasbilidade do condutor nos acidentes').add_to(heatmap_cause_accidents)

# insere o pontos de incidencia de acordo com seus níveis
for latitude, longitude, weight in coordinates_cause_accidents:
  color = colormap(weight)
  folium.PolyLine([(latitude, longitude), (latitude, longitude)],
                  color=color,
                  weight=7, 
                  opacity=0.4).add_to(heatmap_cause_accidents)

heatmap_cause_accidents

---

**Mapa de acidentes por nível de risco:** apresenta o íncide de acidentes de acordo com a normalização realizada nos dados referentes ao tipo de acidente

---

In [ ]:
# lista de coordendas dos acidentes
coordinates_type_accidents = []

# armazena as coordenadas de todos acincentes e seus respectivos tipos registrados no dataframe
for latitude, longitude, weight in zip(df.latitude.values, df.longitude.values, df.tipo_acidente_normalizado.values):    
  coordinates_type_accidents.append([latitude, longitude, weight])

# intancia a visualização do mapa do Nordeste
heatmap_type_accidents = folium.Map(
    location=[-9.231630041999422, -39.87133381935983],
    tiles="Stamen Terrain",
    zoom_start=6,    
)

# define a escala de cores para cada nível risco do acidente
colormap = cm.LinearColormap(colors=['#FDE788', '#FEAF68', '#FE7748', '#FF3F27', '#FF0707'], 
                             index=[0.0, .25, .50, .75, 1.0], 
                             vmin=0.0, 
                             vmax=1.0, 
                             caption='Nível de risco nos acidentes').add_to(heatmap_type_accidents)

# insere o pontos de incidencia de acordo com seus níveis
for latitude, longitude, weight in coordinates_type_accidents:
  color = colormap(weight)
  folium.PolyLine([(latitude, longitude), (latitude, longitude)],
                  color=color,
                  weight=7, 
                  opacity=0.4).add_to(heatmap_type_accidents)

heatmap_type_accidents

---

**Gráfico dos acidentes em cada estado:** apresenta quantos acidentes ocorreram em cada estado 

---

In [ ]:
# lista a quantidade de acidentes por cada estado
accidents_by_state = df["uf"].value_counts()

# gera e plota o gráfico de acidentes em relação a cada estado
accidents_by_state.plot(kind="bar", 
                        figsize=(20,10),                         
                        ylabel='numero de acidentes')
plt.show()

---

**Gráfico dos acidentes em cada BR:** apresenta quantos acidentes ocorreram em cada rodovia federal

---

In [ ]:
# lista a quantidade de acidentes em cada rodovia
accidents_by_road = df["br"].value_counts()

# gera e plota o gráfico de acidentes em relação a cada rodovia
accidents_by_road.plot(kind="bar", 
                        figsize=(20,10),                         
                        ylabel='numero de acidentes')
plt.show()

---

**Gráfico dos acidentes em cada turno:** apresenta quantos acidentes ocorreram em cada turno do dia

---

In [ ]:
# lista todos horarios de acidentes 
hours_accidents = pd.to_datetime(df['horario']).dt.time

# define o período diurno entre 6h até as 18h
start_day = time(6, 0, 0)
finish_day = time(18, 0, 0)

# inicializa os contadores diurno e noturno
day = night = 0

# contabiliza quantos acidentes ocorram em cada turno do dia
for hour_accident in hours_accidents:
  if (hour_accident >= start_day) and (hour_accident <= finish_day):
    day += 1
  else:
    night += 1

# gera e plota o gráfico de acidentes em relação ao turno do dia
labels = "Diurno", "Noturno"
sizes = [day, night]

fig1, ax1 = plt.subplots(figsize=(9, 9))
ax1.pie(sizes,         
        labels=labels,
        autopct='%1.1f%%',
        shadow=True,              
        startangle=90)
ax1.axis('equal')

plt.show()

---

**Gráfico de acidentes em relação a condição climática:** apresenta quantitavamente os acidentes de acordo com a condição metereológica

---

In [ ]:
# lista a quantidade de acidentes ocorridos em cada coondição metereológica
accidents_by_weather_condition = df["condicao_metereologica"].value_counts()

# gera e plota o gráfico de acidentes em relação condição metereológica
accidents_by_weather_condition.plot(kind="bar", 
                                    figsize=(20,10),
                                    ylabel='numero de acidentes')
plt.show()